# Deutsch-Jozsa algorithm

In [ ]:
from qat.lang.AQASM import *

TODO brief recap of alg

Start by definining an abstract oracle

In [ ]:
# This gates accept an int as parameter. It will contain the number of qubits
my_oracle = AbstractGate("MyOracle", [int])

In [ ]:
nqubits = 5


pr = Program()
qr = pr.qalloc(nqubits)
qout = pr.qalloc(1)

pr.apply(X, qout)
pr.apply(H, qout)

for qb in qr:
    pr.apply(H, qb)
pr.apply(my_oracle(nqubits), [*qr, *qout])
for qb in qr:
    pr.apply(H, qb)
pr.apply(H, qout)
pr.apply(X, qout)

In [ ]:
circ = pr.to_circ()
%qatdisplay circ

Exception since we do not have a proper implementation

In [ ]:
from qat.qpus import PyLinalg
qpu = PyLinalg()

In [ ]:
try:
    qpu.submit(circ.to_job())
except:
    print("Exception, as expected")

## Oracles

Define the two oracles

In [ ]:
@build_gate("MyOracle", [int])
def constant_oracle(nqubits):
    qrout = QRoutine()
    qreg = qrout.new_wires(nqubits)
    qout = qrout.new_wires(1)
    return qrout

In [ ]:
@build_gate("MyOracle", [int])
def balanced_oracle(nqubits):
    qrout = QRoutine()
    qreg = qrout.new_wires(nqubits)
    qout = qrout.new_wires(1)
    for qb in qreg:
        qrout.apply(CNOT, qb, qout)
        
    return qrout

Link a proper oracle

In [ ]:
circ = pr.to_circ(link=[constant_oracle])
res = qpu.submit(circ.to_job())
for sample in res:
    print(f"State {sample.state} with amplitude {sample.amplitude}")

or just sampling `qr` and getting probabilities

In [ ]:
circ = pr.to_circ(link=[constant_oracle])
res = qpu.submit(circ.to_job(qubits=[qr]))
for sample in res:
    print(f"State {sample.state} with probability {sample.probability}")

In [ ]:
circ = pr.to_circ(link=[balanced_oracle])
res = qpu.submit(circ.to_job())
for sample in res:
    print(f"State {sample.state} with amplitude {sample.amplitude}")

or just sampling `qr` and getting probabilities

In [ ]:
circ = pr.to_circ(link=[balanced_oracle])
res = qpu.submit(circ.to_job(qubits=[qr]))
for sample in res:
    print(f"State {sample.state} with amplitude {sample.probability}")